In [ ]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np

# FOR PHASE 1 of Training
def lr_schedule_1(epoch): 
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003        
    return lrate

# FOR PHASE 2 of Training
def lr_schedule_2(epoch):  
    lrate = 0.001
    if epoch > 30:
        lrate = 0.0005
    elif epoch > 45:
        lrate = 0.0003        
    return lrate

# FOR PHASE 3 of Training
def lr_schedule_3(epoch): 
    lrate = 0.001
    #if epoch > 15:
    #    lrate = 0.0005
    #elif epoch > 25:
    #    lrate = 0.0003        
    return lrate
    
#Data normalization as per CIFAR-10 dataset instructions - https://www.cs.toronto.edu/~kriz/cifar.html
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)



170500096/170498071 [==============================] - 2s 0us/step


In [ ]:
# Construct CNN

weight_decay = 1e-4
model = Sequential()

#Add convolutional layers
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# Add softmax (output) layer
model.add(Dense(num_classes, activation='softmax'))


#data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

#training
batch_size = 64

opt_rms = keras.optimizers.RMSprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

# Only one model.fit call of the 3 listed below must be uncommented while execution

# For Phase 1: 125 epochs
model.fit((datagen.flow(x_train, y_train, batch_size=batch_size)),steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule_1)])

# For Phase 2: 60 epochs
model.fit((datagen.flow(x_train, y_train, batch_size=batch_size)),steps_per_epoch=x_train.shape[0] // batch_size,epochs=60,verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule_2)])

# For Phase 3: 30 epochs
model.fit((datagen.flow(x_train, y_train, batch_size=batch_size)),steps_per_epoch=x_train.shape[0] // batch_size,epochs=30,verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])



In [ ]:
y_pred_test = model.predict(x_test)
#y_pred_test
joint_prob = np.zeros((10,10)) #for CIFAR 10 (i,j) - state (true label) i, action(prediction prob) j
for i in range(10):
  idx  = np.where(np.argmax(y_test,1)==i)
  for j in range(10):
    joint_prob[i,j] = np.mean(y_pred_test[idx,j])

joint_prob = joint_prob*0.1 # prior pmf over the set of true image classes in CIFAR-10  is uniform.

# joint_prob[x,a] is the joint probability that the true image class is 'x' and predicted image class is 'a'
joint_prob

array([[8.58219564e-02, 1.76765360e-03, 3.17039005e-03, 7.75948493e-04,
        1.00595765e-03, 1.41213543e-04, 1.13032786e-03, 6.73137791e-04,
        3.22457924e-03, 2.28883736e-03],
       [9.57089197e-04, 9.39056873e-02, 3.32127034e-05, 4.88852151e-05,
        5.24008472e-05, 1.80122617e-05, 5.32323075e-04, 9.20354447e-05,
        1.09957848e-03, 3.26077975e-03],
       [5.76674566e-03, 5.95888728e-04, 6.72594130e-02, 2.77847443e-03,
        6.13497160e-03, 2.70892642e-03, 1.14279240e-02, 1.30946450e-03,
        6.78435992e-04, 1.33975660e-03],
       [3.18632200e-03, 1.48408823e-03, 5.41307591e-03, 5.13368547e-02,
        5.84024005e-03, 9.95586142e-03, 1.52682766e-02, 3.63087878e-03,
        1.51082380e-03, 2.37358101e-03],
       [1.81332640e-03, 3.23391124e-04, 5.13699017e-03, 2.41575148e-03,
        7.68427134e-02, 1.02313235e-03, 8.74683559e-03, 2.86123883e-03,
        5.29394438e-04, 3.07227089e-04],
       [1.22920172e-03, 7.08017545e-04, 4.03032191e-03, 1.22082368e-02,
   

In [ ]:
#Code structure credits :  https://github.com/abhijeet3922/Object-recognition-CIFAR-10/blob/master/cifar10_90.py